# Предобученная модель YOLO для детектинга людей  



In [1]:
from ultralytics import YOLO
import cv2

# Загрузка предобученной модели YOLOv8 (можно 'yolov8n.pt', 'yolov8s.pt' и т.д.)
model = YOLO("runs/train/yolov10n_bus_stop_finetuned8/weights/best.pt")

FileNotFoundError: [Errno 2] No such file or directory: 'runs\\train\\yolov10n_bus_stop_finetuned8\\weights\\best.pt'

In [ ]:
# Загрузка изображения
#image = cv2.imread('2.png')
image = cv2.imread('E://AIM/AI-Flow-Detecting/ai-core/data/2025-08-08_11-29-04.png')

# Детекция людей (класс '0' в COCO = человек)
results = model(image, classes=[0])  

# Визуализация результатов
annotated_image = results[0].plot()  # Рисует bounding boxes
cv2.imshow('Detected People', annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Подсчёт количества людей
people_count = len(results[0].boxes)
print(f"Количество людей на фото: {people_count}")



0: 384x640 6 persons, 137.8ms
Speed: 12.2ms preprocess, 137.8ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)
Количество людей на фото: 6


In [ ]:
from ultralytics import YOLO
import cv2

# Загрузка модели YOLOv8 (nano — быстро, подходит для реального времени)
model = YOLO('yolov10n.pt')

# URL видеопотока (убраны лишние пробелы в конце!)
stream_url = "https://restreamer.vms.evo73.ru/918335436b92ac26/stream.m3u8"

# Открываем видеопоток
cap = cv2.VideoCapture(stream_url)

if not cap.isOpened():
    print("❌ Ошибка: Не удалось открыть видеопоток. Проверь URL и соединение.")
    exit()

print("✅ Видеопоток запущен. Детекция людей через YOLOv8...")

while True:
    ret, frame = cap.read()

    if not ret:
        print("⚠️ Не удалось получить кадр. Поток может быть разорван.")
        break

    # Детекция ТОЛЬКО людей (класс 0 в COCO)
    results = model.predict(frame, classes=[0], conf=0.5, verbose=False)

    # Наносим bounding boxes и метки
    annotated_frame = results[0].plot()

    # Подсчёт обнаруженных людей
    people_count = len(results[0].boxes)
    cv2.putText(annotated_frame, f'Людей: {people_count}', (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2, cv2.LINE_AA)

    # Показываем кадр
    cv2.imshow('YOLOv8 — Детекция людей в реальном времени', annotated_frame)

    # Выход по клавише 'q' или пробелу
    if cv2.waitKey(1) & 0xFF in [ord('q'), ord(' ')]:
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()
print("⏹️ Работа завершена.")

✅ Видеопоток запущен. Детекция людей через YOLOv8...


KeyboardInterrupt: 

: 

In [ ]:
import cv2
from ultralytics import YOLO
#plots
import matplotlib.pyplot as plt
import seaborn as sns

#basics
import pandas as pd
import numpy as np
import os
import subprocess

from tqdm.notebook import tqdm

# Display image and videos
import IPython
from IPython.display import Video, display
%matplotlib inline


import urllib.request 
import shutil

In [ ]:
import cv2
import time
import os

os.makedirs("data/images", exist_ok=True)

cap = cv2.VideoCapture("https://restreamer.vms.evo73.ru/918335436b92ac26/stream.m3u8")
count = 0

print("Сбор кадров начат... Нажми 'q' для остановки.")

while True:
    ret, frame = cap.read()
    if ret:
        if count % 60 == 0:  # один кадр каждые 2 сек (30 FPS)
            cv2.imwrite(f"data/images/frame_{count}.jpg", frame)
            print(f"Сохранён кадр {count}")
        count += 1

    if cv2.waitKey(1) & 0xFF == ord('q') or count > 500:  # 500 кадров
        break

cap.release()
cv2.destroyAllWindows()

Сбор кадров начат... Нажми 'q' для остановки.
Сохранён кадр 0
Сохранён кадр 60
Сохранён кадр 120
Сохранён кадр 180
Сохранён кадр 240
Сохранён кадр 300
Сохранён кадр 360
Сохранён кадр 420
Сохранён кадр 480


In [ ]:
# train.py
from ultralytics import YOLO
import torch

# Выбираем устройство
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используется: {device}")

# Загружаем предобученную модель
model = YOLO('yolov8n.pt')  # можно yolov8s.pt для точности

# Обучение
model.train(
    data='data.yaml',        # см. ниже
    epochs=100,
    imgsz=640,
    batch=16,
    device=device,
    name='yolov10n_bus_stop_finetuned',
    augment=True,
    patience=15,
    optimizer='AdamW',
    lr0=1e-3,
    iou=0.5,
    conf=0.25,
    project='runs/train'
)

# Сохранение
model.save('models/yolov10n_best.pt')
print("✅ Модель дообучена и сохранена.")

Используется: cpu
Ultralytics 8.3.176  Python-3.12.6 torch-2.8.0+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=0.25, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.5, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov10n_bus_stop_finetuned, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=15, perspective=0

RuntimeError: Dataset 'data.yaml' error  'data.yaml' does not exist

In [ ]:
# detect_on_stream.py
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd
import os
import subprocess

# -------------------------------
# ### Конфигурации
# -------------------------------
MODEL_PATH = 'runs/train/yolov8n_bus_stop_finetuned/weights/best.pt'  # твоя дообученная модель
STREAM_URL = "https://restreamer.vms.evo73.ru/918335436b92ac26/stream.m3u8"
CONF_LEVEL = 0.6
SCALE_PERCENT = 100
VIDEO_NAME = "result.mp4"
ROI_POINTS = np.array([(1250, 400), (750, 400), (700, 800), (1200, 800)], np.int32)
ALPHA = 0.1
PATIENCE = 100
FRAME_MAX = 5
THR_CENTERS = 25

# -------------------------------
# Загрузка модели
# -------------------------------
model = YOLO(MODEL_PATH)
dict_classes = model.model.names

# -------------------------------
# Открываем поток
# -------------------------------
cap = cv2.VideoCapture(STREAM_URL)
if not cap.isOpened():
    print("❌ Не удалось открыть поток")
    exit()

# Параметры видео
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = cap.get(cv2.CAP_PROP_FPS)

if SCALE_PERCENT != 100:
    width = int(width * SCALE_PERCENT / 100)
    height = int(height * SCALE_PERCENT / 100)

# -------------------------------
# VideoWriter
# -------------------------------
tmp_out = "tmp_result.mp4"
output_out = "rep_result.mp4"

if os.path.exists(tmp_out): os.remove(tmp_out)
if os.path.exists(output_out): os.remove(output_out)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(tmp_out, fourcc, fps, (width, height))

# -------------------------------
# Вспомогательные функции
# -------------------------------
def resize_frame(frame, scale):
    if scale == 100: return frame
    w = int(frame.shape[1] * scale / 100)
    h = int(frame.shape[0] * scale / 100)
    return cv2.resize(frame, (w, h), interpolation=cv2.INTER_AREA)

def update_tracking(centers_old, center, thr, last_key, frame_idx, frame_max):
    best_id = None
    min_dist = float('inf')
    for obj_id, history in centers_old.items():
        last_center = list(history.keys())[-1]
        dist = np.linalg.norm(np.array(last_center) - np.array(center))
        if dist < thr and dist < min_dist:
            min_dist = dist
            best_id = obj_id

    if best_id is not None:
        centers_old[best_id][center] = frame_idx
        return centers_old, best_id, False, best_id
    else:
        new_id = f"p_{len(centers_old)}"
        centers_old[new_id] = {center: frame_idx}
        return centers_old, new_id, True, new_id

def filter_tracks(centers_old, patience):
    current_frame = max([max(t.values()) for t in centers_old.values()] + [0])
    to_remove = [k for k, v in centers_old.items() if current_frame - max(v.values()) > patience]
    for k in to_remove:
        del centers_old[k]
    return centers_old

# -------------------------------
# Основной цикл
# -------------------------------
centers_old = {}
total_count = 0
frame_idx = 0

print("✅ Обработка начата. Нажмите 'q' для остановки.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("⚠️ Кадр не получен. Переподключение...")
        cap.release()
        cap = cv2.VideoCapture(STREAM_URL)
        continue

    frame_idx += 1
    frame = resize_frame(frame, SCALE_PERCENT)
    display_frame = frame.copy()

    # --- ROI ---
    overlay = display_frame.copy()
    cv2.polylines(overlay, [ROI_POINTS], True, (255, 0, 0), 2)
    cv2.fillPoly(overlay, [ROI_POINTS], (255, 0, 0))
    cv2.addWeighted(overlay, ALPHA, display_frame, 1 - ALPHA, 0, display_frame)

    # Вырезаем ROI
    x_min, y_min = ROI_POINTS[:, 0].min(), ROI_POINTS[:, 1].min()
    x_max, y_max = ROI_POINTS[:, 0].max(), ROI_POINTS[:, 1].max()
    roi_frame = frame[y_min:y_max, x_min:x_max]

    # --- Детекция ---
    results = model(roi_frame, conf=CONF_LEVEL, classes=[0], verbose=False)
    if len(results[0].boxes) == 0:
        detections = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'conf'])
    else:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        conf = results[0].boxes.conf.cpu().numpy()
        detections = pd.DataFrame(np.column_stack([boxes, conf]),
                                  columns=['xmin', 'ymin', 'xmax', 'ymax', 'conf'])

    # --- Обработка каждого человека ---
    for _, row in detections.iterrows():
        xmin, ymin, xmax, ymax, conf_val = map(int, row)
        cx = (xmin + xmax) // 2 + x_min
        cy = (ymin + ymax) // 2 + y_min

        centers_old, obj_id, is_new, _ = update_tracking(
            centers_old, (cx, cy), THR_CENTERS, None, frame_idx, FRAME_MAX
        )
        total_count += is_new

        # Рисуем
        cv2.rectangle(display_frame, (xmin + x_min, ymin + y_min), (xmax + x_min, ymax + y_min), (0, 255, 0), 2)
        cv2.circle(display_frame, (cx, cy), 5, (0, 255, 0), -1)
        cv2.putText(display_frame, f"{obj_id} ({conf_val:.2f})", (xmin + x_min, ymin + y_min - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)

    # Фильтрация
    centers_old = filter_tracks(centers_old, PATIENCE)

    # Счётчик
    cv2.putText(display_frame, f"Людей в зоне: {total_count}", (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2)

    # Запись и отображение
    out.write(display_frame)
    cv2.imshow("YOLOv8 — Пассажиропоток", display_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# -------------------------------
# Финализация
# -------------------------------
out.release()
cap.release()
cv2.destroyAllWindows()

# Перекодировка
subprocess.run([
    "ffmpeg", "-i", tmp_out,
    "-crf", "18", "-preset", "veryfast",
    "-vcodec", "libx264", output_out
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

os.remove(tmp_out)
print(f"✅ Видео сохранено: {output_out}")

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd
import os
import subprocess
from tqdm import tqdm

# -------------------------------
# ### Configurations
# -------------------------------
verbose = False
scale_percent = 100  # масштабирование кадра
conf_level = 0.8     # порог уверенности
thr_centers = 20     # порог для сопоставления центров
frame_max = 5        # макс. кадров до "потери" объекта
patience = 100       # макс. длина истории треков
alpha = 0.1          # прозрачность ROI
video_name = 'result.mp4'

# URL видеопотока
stream_url = "https://restreamer.vms.evo73.ru/918335436b92ac26/stream.m3u8"

# Загрузка модели
model = YOLO('yolov8n.pt')  # или yolov10n.pt

# Классы: только люди
class_IDS = [0]
dict_classes = {0: 'person'}

# Вспомогательные переменные
centers_old = {}
obj_id = 0
count_p = 0
lastKey = ''

# -------------------------------
# Открываем видеопоток
# -------------------------------
cap = cv2.VideoCapture(stream_url)

if not cap.isOpened():
    print("❌ Ошибка: Не удалось открыть видеопоток.")
    exit()

# Параметры видео
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = cap.get(cv2.CAP_PROP_FPS)

print(f'[INFO] - Original Dim: {width}x{height}, FPS: {fps}')

# Масштабирование (если нужно)
if scale_percent != 100:
    width = int(width * scale_percent / 100)
    height = int(height * scale_percent / 100)
    print(f'[INFO] - Dim Scaled: {width}x{height}')

# Функция для изменения размера кадра
def risize_frame(frame, scale_percent):
    if scale_percent == 100:
        return frame
    width_new = int(frame.shape[1] * scale_percent / 100)
    height_new = int(frame.shape[0] * scale_percent / 100)
    return cv2.resize(frame, (width_new, height_new), interpolation=cv2.INTER_AREA)

# -------------------------------
# Настройка записи видео
# -------------------------------
output_path = "rep_" + video_name
tmp_output_path = "tmp_" + output_path

# Удаляем старые файлы
for f in [tmp_output_path, output_path]:
    if os.path.exists(f):
        os.remove(f)

VIDEO_CODEC = cv2.VideoWriter_fourcc(*"MP4V")
output_video = cv2.VideoWriter(tmp_output_path, VIDEO_CODEC, fps, (width, height))

if not output_video.isOpened():
    print("❌ Ошибка: Не удалось инициализировать VideoWriter.")
    cap.release()
    exit()

# -------------------------------
# ROI: область интереса (остановка)
# -------------------------------
def get_roi_points():
    # Пример: треугольник или трапеция на остановке
    # Адаптируй под свою камеру!
    return [np.array([(1250, 400), (750, 400), (700, 800), (1200, 800)], np.int32)]

# -------------------------------
# Функция обновления трекинга
# -------------------------------
def update_tracking(centers_old, current_center, threshold, lastKey, frame_idx, frame_max):
    center_x, center_y = current_center
    is_new = False
    min_dist = float('inf')
    best_key = None

    for obj_id, centers in centers_old.items():
        # Последний известный центр
        last_center = list(centers.keys())[-1]
        cx, cy = last_center
        dist = np.sqrt((cx - center_x)**2 + (cy - center_y)**2)
        if dist < min_dist and dist < threshold:
            min_dist = dist
            best_key = obj_id

    if best_key is not None:
        # Обновляем трек
        centers_old[best_key][(center_x, center_y)] = frame_idx
        return centers_old, best_key, False, best_key
    else:
        # Новый объект
        new_id = f"person_{obj_id}"
        centers_old[new_id] = {(center_x, center_y): frame_idx}
        return centers_old, new_id, True, new_id

# -------------------------------
# Фильтрация старых треков
# -------------------------------
def filter_tracks(centers_old, patience):
    current_frame = max([max(frames.values()) for frames in centers_old.values()] + [0]) if centers_old else 0
    keys_to_remove = []
    for obj_id, centers in centers_old.items():
        last_seen = max(centers.values())
        if current_frame - last_seen > patience:
            keys_to_remove.append(obj_id)
    for k in keys_to_remove:
        del centers_old[k]
    return centers_old

# -------------------------------
# Основной цикл обработки потока
# -------------------------------
print("✅ Начало обработки видеопотока. Нажмите 'q' для остановки.")

frame_count = 0
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("⚠️ Не удалось получить кадр. Попытка переподключения...")
            time.sleep(2)
            cap.release()
            cap = cv2.VideoCapture(stream_url)
            continue

        frame_count += 1
        frame = risize_frame(frame, scale_percent)
        overlay = frame.copy()

        # --- ROI ---
        area_roi = get_roi_points()
        cv2.polylines(overlay, pts=area_roi, isClosed=True, color=(255, 0, 0), thickness=2)
        cv2.fillPoly(overlay, area_roi, (255, 0, 0))
        frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

        # Вырезаем ROI
        x_min = min([p[0] for p in area_roi[0]])
        x_max = max([p[0] for p in area_roi[0]])
        y_min = min([p[1] for p in area_roi[0]])
        y_max = max([p[1] for p in area_roi[0]])
        roi_frame = frame[y_min:y_max, x_min:x_max]

        # --- Детекция ---
        results = model.predict(roi_frame, conf=conf_level, classes=class_IDS, verbose=False)
        if len(results[0].boxes) == 0:
            positions_frame = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'conf', 'class'])
        else:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            conf = results[0].boxes.conf.cpu().numpy()
            classes = results[0].boxes.cls.cpu().numpy()
            positions_frame = pd.DataFrame(np.column_stack([boxes, conf, classes]),
                                           columns=['xmin', 'ymin', 'xmax', 'ymax', 'conf', 'class'])

        # --- Обработка каждого человека ---
        for idx, row in positions_frame.iterrows():
            xmin, ymin, xmax, ymax, conf_val, cls = map(int, row[:6])
            center_x = (xmin + xmax) // 2
            center_y = (ymin + ymax) // 2

            # Смещение центра в глобальные координаты
            center_x += x_min
            center_y += y_min

            centers_old, obj_id, is_new, lastKey = update_tracking(
                centers_old, (center_x, center_y), thr_centers, lastKey, frame_count, frame_max
            )
            count_p += is_new

            # Рисуем bounding box и центр
            cv2.rectangle(frame, (xmin + x_min, ymin + y_min), (xmax + x_min, ymax + y_min), (0, 0, 255), 2)
            cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)
            cv2.putText(frame, f"{obj_id}:{conf_val:.2f}", (xmin + x_min, ymin + y_min - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1)

        # --- Фильтрация старых треков ---
        centers_old = filter_tracks(centers_old, patience)

        # --- Отображение счётчика ---
        cv2.putText(frame, f'People in ROI: {count_p}', (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 2)

        # --- Запись кадра ---
        output_video.write(frame)

        # --- Показываем результат ---
        cv2.imshow("Live Detection", frame)
        if cv2.waitKey(1) & 0xFF in [ord('q'), ord(' ')]:
            break

except KeyboardInterrupt:
    print("⏹️ Остановлено пользователем.")

# -------------------------------
# Финализация: перекодировка видео
# -------------------------------
output_video.release()
cap.release()
cv2.destroyAllWindows()

print("🔁 Перекодировка видео для совместимости...")

if os.path.exists(output_path):
    os.remove(output_path)

subprocess.run([
    "ffmpeg", "-i", tmp_output_path,
    "-crf", "18",
    "-preset", "veryfast",
    "-hide_banner", "-loglevel", "error",
    "-vcodec", "libx264", output_path
], check=True)

os.remove(tmp_output_path)
print(f"✅ Видео сохранено: {output_path}")

[INFO] - Original Dim: 1920x1080, FPS: 25.0
✅ Начало обработки видеопотока. Нажмите 'q' для остановки.


UnboundLocalError: cannot access local variable 'obj_id' where it is not associated with a value

In [ ]:
# Checking samples of processed frames
for i in [62,63, 64, 65, 66]:
    plt.figure(figsize =( 14, 10))
    plt.imshow(frames_list[i])
    plt.show()

In [ ]:
#output video result
frac = 0.7 
Video(data='rep_result.mp4', embed=True, height=int(720 * frac), width=int(1280 * frac))